<a href="https://colab.research.google.com/github/HENILCHOPRA/AdversarialNetsPapers/blob/master/DGAZE_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Dataset and back up to drive

In [ ]:
!wget http://cvit.iiit.ac.in/images/Projects/DGAZE/DGAZE_dataset.zip

In [ ]:
!unzip -P "eZaGd!@#" "/content/DGAZE_dataset.zip"


Archive:  /content/DGAZE_dataset.zip
 extracting: dataset_download.zip    


In [ ]:
!unzip -P "eZaGd!@#" "/content/dataset_download.zip"

In [ ]:
!zip -r "/content/DGAZE_data.zip" "/content/dataset_download"

In [ ]:
!cp -r "/content/DGAZE_data.zip" -d "/content/drive/MyDrive/Colab/Data/DGAZE"

# Preprocess

In [ ]:
remo

import dlib
from tqdm import tqdm 


In [ ]:
os.makedirs(os.path.join('/content', "preprocessed"))
os.makedirs(os.path.join('/content/preprocessed', "targets"))

In [ ]:
#convert txt file to target coordinates for each video files
import os
arr = os.listdir("/content/dataset_download/road_view")
for fl in arr:
  if "txt" in fl:
    data = pd.read_csv('/content/dataset_download/road_view/trip100_out.txt', sep=",", header=None)
    data.columns = ["frame", "x1", "y1", "x2","y2"]
    data.drop(["frame"], axis=1).to_csv("/content/preprocessed/targets/"+fl.split(".")[0]+".csv")

In [ ]:
#get subfolder names
arr = os.listdir("/content/dataset_download/driver10/driver_view")
for i in range(len(arr)):
  arr[i] = arr[i].split('.')[0]
subfolder_names = arr

In [ ]:
#create dataset folders
preprocessed_base = "/content/preprocessed/"
arr = os.listdir("/content/dataset_download")
driver_fols = []
for fl in arr:
  if 'driver' in fl:
    driver_fols.append(fl)

for fol in driver_fols:
  path = os.path.join(preprocessed_base, fol)
  os.makedirs(path)
  for sub_fol in subfolder_names:
    os.makedirs(os.path.join(path, sub_fol))

In [ ]:
def get_frame_count(input_video):
    '''
    func to count number of frames in input video
    ''' 
    count = 0
    vid = cv2.VideoCapture(input_video)
    while(True):
        ret, frame = vid.read()
        if(ret == False):
            break
        count += 1
    return count


        
def get_align_videos(path, driver, sample_no, driver_view_out, road_view_out, drop):
    '''
    func to obtain road view by matching frames from driver view, 
    road view and projected road view to get the same number of frames in each.
    steps:
        -- get number of frames in driver view, projected road view and actual road view
        -- Drop extra frames using (drop = larger_frame_count/diff) to get equal number of frames in each video
    '''
    driver_view = os.path.join(path, 'dataset_download', driver, 'driver_view', 'sample' +\
                                                                       str(sample_no) +'.avi')            
    road_view = os.path.join(path,  'dataset_download', 'road_view', 'trip' + str(sample_no)\
                                                                     + '_out_hist.avi')
              
    if sample_no < 10:
        gt_point = os.path.join(path, 'dataset_download', 'road_view', 'trip' + str(sample_no) + '_out.npy')
        gt_point = np.load(gt_point)
    else:
        gt_point = os.path.join(path, 'dataset_download', 'road_view', 'trip' + str(sample_no) + '_out.txt')
        gt_point = pd.read_csv(gt_point, header = None).values   
                         
    cap_driver_view = cv2.VideoCapture(driver_view) # read driver view
    nframes_driver_view = get_frame_count(driver_view) # frame count of driver view video
    
    cap_road_view = cv2.VideoCapture(road_view) # read road view
    nframes_road_view = get_frame_count(road_view) # frame count of road view video
   
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    
    driver_view_out = os.path.join(path, 'dataset', driver, 'driver_view', 'sample' + str(sample_no) +'.avi')
    driver_view_out = cv2.VideoWriter(driver_view_out, fourcc, 25, (1440,1080)) 
    
    road_view_out = os.path.join(path,  'dataset', driver, 'road_view', 'sample' + str(sample_no) +'.avi')
    road_view_out = cv2.VideoWriter(road_view_out, fourcc, 25, (1920, 1080)) 
    
    road_gaze_point_out = os.path.join(path,  'dataset', driver, 'road_view', 'sample' + str(sample_no) +'.npy')
    
    i = 0; count = 0; k = 0; gaze_point_out = []

    nframes = nframes_driver_view
    
    while(i < nframes):
        i += 1     
        if(drop != -1 and i%drop == 0):
            if(nframes_driver_view > nframes_road_view):
                ret1, frame1 = cap_driver_view.read()
            else:
                ret3, frame3 = cap_road_view.read()
                k +=1
        else:
            ret1, frame1 = cap_driver_view.read()
            ret3, frame3 = cap_road_view.read()
            if(ret1 == False or ret3 == False):
                break
            gaze_point_out.append(gt_point[k])
            k +=1
            count += 1
            driver_view_out.write(frame1)
            road_view_out.write(frame3)
            
              
    gaze_point_out = np.array(gaze_point_out)
    np.save(road_gaze_point_out, gaze_point_out)
        
    driver_view_out.release(); road_view_out.release();
    cap_driver_view.release(); cap_road_view.release()
             

        
def dataset(path):
    
    assert os.path.exists(path), "path does not exists!"
        
    drivers = os.listdir(path + 'dataset_download/')
    
    for driver in drivers[:5]:
        
        if driver == 'road_view':
            continue;
        
        print(f'--> Processing for {driver}')
        
        nsamples = len(os.listdir(os.path.join(path, 'dataset_download', driver, 'driver_view')))
        
        driver_view_out = os.path.join(path, 'dataset', driver, 'driver_view')
        road_view_out = os.path.join(path, 'dataset', driver, 'road_view')
        
        if os.path.exists(driver_view_out) and os.path.exists(road_view_out):
            continue
        
        if not os.path.exists(driver_view_out):
            os.makedirs(driver_view_out, exist_ok=True)
        
        if not os.path.exists(road_view_out):
            os.makedirs(road_view_out, exist_ok=True)

        drop_data = pd.read_csv(os.path.join(path, 'dataset_download', driver, 'drop_rate_per_video.csv'))
        samples = drop_data['sample_no']
        drop_rate = drop_data['drop_rate'].astype(int)

        for sample_no in tqdm(range(1, nsamples+1)):
            drop = drop_rate[np.where(samples == 'sample'+str(sample_no)+'.avi')[0][0]]
            get_align_videos(path, driver, sample_no, driver_view_out, road_view_out, drop)

if __name__ == '__main__':
    path = "/content/"
    dataset(path)

--> Processing for driver18


100%|██████████| 112/112 [15:19<00:00,  8.21s/it]


--> Processing for driver24


100%|██████████| 112/112 [15:13<00:00,  8.16s/it]


--> Processing for driver11


100%|██████████| 112/112 [13:39<00:00,  7.32s/it]


--> Processing for driver10


100%|██████████| 112/112 [13:06<00:00,  7.03s/it]


In [ ]:
!zip -r "/content/drive/MyDrive/Colab/Data/DGAZE_data.zip" "/content/dataset"

In [ ]:
!unzip '/content/drive/MyDrive/Colab/Data/DGAZE_data.zip' -d '/content/dataset'

In [ ]:
import os
lst = os.listdir("/content/dataset/content/dataset")
frames_base = '/content/frames'

for fol in lst:
  os.makedirs(os.path.join(frames_base, fol))


In [ ]:
frame_count = 0
df_target = pd.DataFrame(columns=[0,1])

def vid_to_frame(input_pth):
  global frame_count
  vidcap = cv2.VideoCapture(input_pth)
  success,image = vidcap.read()
  while success:
    cv2.imwrite("/content/frames/%d.jpg" % frame_count, image)     # save frame as JPEG file      
    success,image = vidcap.read()
    frame_count += 1

def process_targets(pth, flag = False):
  global df_target
  arr = np.load(pth, allow_pickle= True)
  df = pd.DataFrame(arr)
  if flag:
    df_target = df_target.append(df[[0, 1]])
  else:
    df[0] = (df[1] + df[3]) / 2
    df[1] = (df[2] + df[4]) / 2
    df_target = df_target.append(df[[0,1]])
  


In [ ]:
dri_base = "/content/dataset/content/dataset"
dri_fols = os.listdir(dri_base)

for fol in dri_fols:
  dri_pth = os.path.join(dri_base, fol + '/driver_view')
  tar_pth = os.path.join(dri_base, fol + '/road_view')
  vid_lst = os.listdir(dri_pth)
  for vid in vid_lst:
    if 'avi' in vid:
      print('Processing Target: ', dri_pth+vid)
      vid_id = vid.split('.')[0]
      vid_to_frame(os.path.join(dri_pth, vid))
      process_targets(os.path.join(tar_pth, vid_id+'.npy'), int(vid_id[6:])<=9)

Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample47.avi
Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample51.avi
Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample43.avi
Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample111.avi
Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample94.avi
Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample18.avi
Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample46.avi
Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample65.avi
Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample62.avi
Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample9.avi
Processing Target:  /content/dataset/content/dataset/driver11/driver_viewsample5.avi
Processing Target:  /content/dataset/content/dataset/dr

In [ ]:
process_targets("/content/dataset/content/dataset/driver11/road_view/sample10.npy", flag= False)

In [ ]:
vid = 'sample112.npy'
vid_id = vid.split('.')[0] 
int(vid_id[6:])<=9

False

In [ ]:
np.load("/content/dataset/content/dataset/driver11/road_view/sample61.npy", allow_pickle= True)
# Driver11: 112.npy

array([], dtype=float64)

In [ ]:
lst = os.listdir('/content/dataset/content/dataset/driver10/road_view')
for fl in lst:
  if 'npy' in fl:
    if len(np.load('/content/dataset/content/dataset/driver10/road_view/' + fl, allow_pickle=True)) == 0:
      print(fl) 

sample98.npy
sample66.npy
sample57.npy
sample65.npy


In [ ]:
sam11 = ['sample112.avi',
'sample19.avi',
'sample57.avi',
'sample61.avi',
'sample64.avi',
'sample88.avi']
sam10 = ['sample98.npy',
'sample66.npy',
'sample57.npy',
'sample65.npy']
sam = '/content/dataset/content/dataset/driver11/driver_view/'
sam = '/content/dataset/content/dataset/driver10/driver_view/'
for i in sam10:
  try:
    os.remove(sam + i)
  except:
    print

In [ ]:
lst = os.listdir('/content/frames')
len(lst)

45511

In [ ]:
!zip -r "/content/drive/MyDrive/Colab/Data/DGAZE_data.zip" "/content/frames"

Streaming output truncated to the last 5000 lines.
  adding: content/frames/41581.jpg (deflated 0%)
  adding: content/frames/43240.jpg (deflated 0%)
  adding: content/frames/4320.jpg (deflated 0%)
  adding: content/frames/41180.jpg (deflated 4%)
  adding: content/frames/37226.jpg (deflated 0%)
  adding: content/frames/119.jpg (deflated 0%)
  adding: content/frames/28030.jpg (deflated 1%)
  adding: content/frames/14312.jpg (deflated 0%)
  adding: content/frames/18952.jpg (deflated 0%)
  adding: content/frames/42244.jpg (deflated 8%)
  adding: content/frames/44822.jpg (deflated 0%)
  adding: content/frames/36298.jpg (deflated 0%)
  adding: content/frames/23787.jpg (deflated 0%)
  adding: content/frames/30184.jpg (deflated 0%)
  adding: content/frames/39743.jpg (deflated 0%)
  adding: content/frames/20160.jpg (deflated 0%)
  adding: content/frames/6480.jpg (deflated 0%)
  adding: content/frames/12666.jpg (deflated 0%)
  adding: content/frames/14952.jpg (deflated 0%)
  adding: content/fram

In [ ]:
df_target.to_csv("/content/drive/MyDrive/Colab/Data/DGAZE_data.csv")